In [1]:
""" Analysis of Museum entites in 'luoghi della cultura dataset' """

from rdflib import Graph
import csv
import pandas as pd
from IPython.display import display

# Load the  datastet luoghi della cultura RDF file
rdf_file =  r"C:\Users\HWRUser\Desktop\DHDK Course\Information Visualization\Data_Italian_publishers\dataset_luoghi_cultura.rdf"
g = Graph()
g.parse(rdf_file, format="xml") 


query_1 ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX clvapit: <https://w3id.org/italia/onto/CLV/>
PREFIX geo1: <http://www.w3.org/2003/01/geo/wgs84_pos#>


SELECT ?museum ?museumlabel ?museumCity ?museumRegion ?museumlatitude ?museumlongitude 
WHERE {
  ?museum rdf:type cis:CulturalInstituteOrSite .
  ?museum dc:type "Museo, Galleria e/o raccolta".
  ?museum rdfs:label ?museumlabel .
  ?museum cis:hasSite ?museumSite.
  ?museumSite cis:siteAddress ?museumAddress.
  ?museumAddress clvapit:hasRegion ?museumRegion.
  ?museumAddress clvapit:hasCity ?museumCity.
  ?museum geo1:lat ?museumlatitude.
  ?museum geo1:long ?museumlongitude.

}
"""


# Perform the query
results = g.query(query_1)

#Museums DataFrame
data_Museums = []
for row in results:
    data_Museums.append({'Museum_ID': str(row['museum']), 'Museum_Name': str(row['museumlabel']), 'Museum_City': str(row['museumCity']), 'Museum_Region': str(row['museumRegion']), 'Museum_Latitude': str(row['museumlatitude']), 'Museum_Longitude': str(row['museumlongitude']) })

df_Museums = pd.DataFrame(data_Museums)
""" Enriching df_Museums dataframe with other Museums entities from OtherPlaces dataframe in order to not leave out any entities 
that can be considered as a Museum"""

df_Other_Places  = pd.read_csv(r'C:\Users\HWRUser\Documents\GitHub\bookdataviz\data\OtherPlaces.csv')
filtered_Other_Places_df = df_Other_Places[df_Other_Places['OtherPlace_Name'].str.contains('muse', case=False, na=False)]

#Renaming columns of new filtered Museum Dataframe
df_filtered_museums=filtered_Other_Places_df.rename(columns={
    'OtherPlace_ID': 'Museum_ID',
    'OtherPlace_Name': 'Museum_Name',
    'OtherPlace_City': 'Museum_City',
    'OtherPlace_Region': 'Museum_Region',
    'OtherPlace_Latitude': 'Museum_Latitude',
    'OtherPlace_Longitude': 'Museum_Longitude'
})

#Dropping "OtherPlace_Type column which is not needed in the new filtered Museum Dataframe"
df_filtered_museums = df_filtered_museums.drop(columns=['OtherPlace_Type'])


#Mergin the previous Museum Dataframe with the filtered Museum Dataframes to create a new Dataframe with all museum entities
df_Museums = pd.DataFrame(data_Museums)
df_museums_complete= pd.concat([df_filtered_museums, df_Museums], ignore_index=True)

#New dataframes for counting museum cities and regions
museum_region_counts_complete = df_museums_complete.groupby('Museum_Region').size().reset_index(name='Count')
museum_city_counts_complete = df_museums_complete.groupby('Museum_City').size().reset_index(name='Count')

#Displaying all dataframes for Museum entities
display(df_museums_complete)
display(museum_region_counts_complete)
display(museum_city_counts_complete)



# #CSV files for Museums from the above Dataframes
csv_file_museums_complete = "Museums_complete.csv"
df_museums_complete.to_csv(csv_file_museums_complete, index=False, encoding='utf-8')
print(f"Results have been written to {csv_file_museums_complete}")

csv_file_museums_region_count_complete = "Museums_region_count_complete.csv"
museum_region_counts_complete.to_csv(csv_file_museums_region_count_complete, index=False, encoding='utf-8')
print(f"Results have been written to {csv_file_museums_region_count_complete}")

csv_file_museums_city_count_complete = "Museums_city_count_complete.csv"
museum_city_counts_complete.to_csv(csv_file_museums_city_count_complete, index=False, encoding='utf-8')
print(f"Results have been written to {csv_file_museums_city_count_complete}")

,Museum_ID,Museum_Name,Museum_City,Museum_Region,Museum_Latitude,Museum_Longitude
0,http://dati.beniculturali.it/mibact/luoghi/res...,Museo Annibale Di Francia,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,38.18351,15.550492
1,http://dati.beniculturali.it/mibact/luoghi/res...,Museo della Fondazione Whitaker,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,37.866444,12.468154
2,http://dati.beniculturali.it/mibact/luoghi/res...,Parco Poesia Pascoli - Museo Casa Pascoli,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,44.105186,12.415602
3,http://dati.beniculturali.it/mibact/luoghi/res...,Museo di arte contemporanea Giuseppe e Titina ...,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,44.868443,9.266259
4,http://dati.beniculturali.it/mibact/luoghi/res...,Museo armi antiche Fosco Baboni,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,45.149033,10.64564
...,...,...,...,...,...,...
4554,http://dati.beniculturali.it/mibact/luoghi/res...,Museo della satira e della caricatura,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,43.959473,10.169657
4555,http://dati.beniculturali.it/mibact/luoghi/res...,Museo Fortuny,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,45.43519,12.335712
4556,http://dati.beniculturali.it/mibact/luoghi/res...,Pinacoteca e Museo delle Arti,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,38.236107,16.265602
4557,http://dati.beniculturali.it/mibact/luoghi/res...,Pinacoteca dei Padri Cappuccini,http://dati.beniculturali.it/mibact/luoghi/res...,http://dati.beniculturali.it/mibact/luoghi/res...,44.615677,8.843204


,Museum_Region,Count
0,http://dati.beniculturali.it/mibact/luoghi/res...,128
1,http://dati.beniculturali.it/mibact/luoghi/res...,63
2,http://dati.beniculturali.it/mibact/luoghi/res...,213
3,http://dati.beniculturali.it/mibact/luoghi/res...,230
4,http://dati.beniculturali.it/mibact/luoghi/res...,490
5,http://dati.beniculturali.it/mibact/luoghi/res...,150
6,http://dati.beniculturali.it/mibact/luoghi/res...,308
7,http://dati.beniculturali.it/mibact/luoghi/res...,180
8,http://dati.beniculturali.it/mibact/luoghi/res...,461
9,http://dati.beniculturali.it/mibact/luoghi/res...,240


,Museum_City,Count
0,http://dati.beniculturali.it/mibact/luoghi/res...,1
1,http://dati.beniculturali.it/mibact/luoghi/res...,1
2,http://dati.beniculturali.it/mibact/luoghi/res...,2
3,http://dati.beniculturali.it/mibact/luoghi/res...,1
4,http://dati.beniculturali.it/mibact/luoghi/res...,2
...,...,...
2241,http://dati.beniculturali.it/mibact/luoghi/res...,1
2242,http://dati.beniculturali.it/mibact/luoghi/res...,1
2243,http://dati.beniculturali.it/mibact/luoghi/res...,1
2244,http://dati.beniculturali.it/mibact/luoghi/res...,1


Results have been written to Museums_complete.csv
Results have been written to Museums_region_count_complete.csv
Results have been written to Museums_city_count_complete.csv
